## 6.2 循环神经网络

### 6.2.1 不含隐藏状态的神经网络

> 1. **给定样本数为n,特征数为d的小批量数据样本  $\boldsymbol{X} \in \mathbb{R}^{n \times d}$,设隐藏层的激活函数$\phi$,则隐藏层的输出$\boldsymbol{H} \in \mathbb{R}^{n \times d}$;**
$$\boldsymbol{H}=\phi(\boldsymbol{X}\boldsymbol{W}_{xh} + \boldsymbol{b}_{h})$$
**其中隐藏层权重参数$\boldsymbol{W}_{xh} \in \mathbb{R}^{n \times d}$; 隐藏层偏差参数$\boldsymbol{b}_{h} \in \mathbb{R}^{1 \times h}$**

> 2. **把隐层变量$\boldsymbol{H}$作为输出层的输入,且输出个数各位为q(如分类),则输出层的输出为**
$$\boldsymbol{O}=\boldsymbol{H}\boldsymbol{W}_{hq} + \boldsymbol{b}_{h}$$
**其中$\boldsymbol{O} \in \mathbb{R}^{n \times q}$;权重参数$\boldsymbol{W}_{hq} \in \mathbb{R}^{h \times q}$;输出层偏差参数$\boldsymbol{b}_q \in \mathbb{R}^{1 \times q}$;如果分类,可通过`softmax`函数**

### 6.2.2 含隐藏状态的神经网络

> 1. **考虑数据存在时间相关性问题,假设$\boldsymbol{X}_{t} \in \mathbb{R}^{n \times d}$是序列中时间步t的小批量输入, $\boldsymbol{H}_{t} \in \mathbb{R}^{n \times h}$为该时间步的隐藏遍历**

> 2. **同时保存上一时间步的隐藏变量$\boldsymbol{H}_{t-1}$,并引入新的权重参数$\boldsymbol{W}_{hh} \in \mathbb{R}^{h \times h}$;所以`时间步t的隐藏变量`由`当前时间步的输入`和`上一时间步的隐藏变量`决定**
$$\boldsymbol{H}_{t} = \phi(\boldsymbol{X}_{t}\boldsymbol{W}_{xh} + \boldsymbol{H}_{t-1}\boldsymbol{W}_{hh} + \boldsymbol{b}_{h})$$
**这里隐藏变量能够捕捉截止当前的时间步的序列的历史信息**

> 3. **时间步t输出为**
$$\boldsymbol{O}_{t} = \boldsymbol{H}_{t}\boldsymbol{W}_{hq} + \boldsymbol{b}_{q}$$
    - **参数包括隐藏层权重$\boldsymbol{W}_{xh} \in \mathbb{R}^{d \times h}$, $\boldsymbol{W}_{hh} \in \mathbb{R}^{h \times h}$和偏差$\boldsymbol{b}_{h} \in \mathbb{R}^{1 \times h}$** 
    - **输出层的权重$\boldsymbol{W}_{hq} \in \mathbb{R}^{h \times q}$和偏差$\boldsymbol{b}_{q} \in \mathbb{R}^{1 \times q}$**
    - **在不同的时间步,`RNN`也使用这些参数,因此`RNN`的参数量不随时间步的增加而增加**
    
> 4. **下图为3个相邻时间步的计算逻辑**
    - **时间步t,隐藏状态的计算可以看做:输入$\boldsymbol{X}_{t}$和前一时间步隐藏状态$\boldsymbol{H}_{t-1}$,然后使用激活函数$\phi$的全连接层**
    - **上一步全连接层的输出就是当前时间步的隐藏状态$\boldsymbol{H}_{t}$,且模型参数为$\boldsymbol{W}_{xh}$与$\boldsymbol{W}_{hh}$的连接,且偏差为$\boldsymbol{b}_{h}$**
    - **当前时间步隐藏状态$\boldsymbol{H}_{t}$参与下一个时间步的计算$\boldsymbol{H}_{t+1}$**
    - **$\boldsymbol{X}_{t}\boldsymbol{W}_{xh} + \boldsymbol{H}_{t-1}\boldsymbol{W}_{hh}$的计算等价于$\boldsymbol{X}_{t}$与$\boldsymbol{H}_{t-1}$连接后的矩阵乘以$\boldsymbol{W}_{xh}$与$\boldsymbol{W}_{hh}$连接后的矩阵**

![avatar](img/6.2_rnn.svg)

In [1]:
# 构造矩阵, X, W_xh, H, W_hh 形状分别为 (3, 1) (1, 4) (3, 4) (4, 4)
# 将X 与 W_xh 、 H 与 W_hh 分别相城和相加, 得到形状为(3, 4)
import torch

X, W_xh = torch.rand(3, 1), torch.rand(1, 4)
H, W_hh = torch.rand(3, 4), torch.rand(4, 4)

torch.matmul(X, W_xh) + torch.matmul(H, W_hh)

tensor([[1.5032, 2.0524, 2.1265, 1.8598],
        [0.9673, 1.1914, 1.3616, 1.7550],
        [1.2635, 1.5883, 2.0362, 1.5591]])

In [2]:
# 将 X 和 H 按列(dim=1)连接, 形状为(3, 5)   5为 1+4
# 将 W_xh 和 W_hh 按行(dim=0)连接, 形状为(5, 4)
torch.matmul(torch.cat((X, H), dim=1), torch.cat((W_xh, W_hh), dim=0))

tensor([[1.5032, 2.0524, 2.1265, 1.8598],
        [0.9673, 1.1914, 1.3616, 1.7550],
        [1.2635, 1.5883, 2.0362, 1.5591]])

### 6.2.3 应用: 基于字符级循环神经网络的语言模型

> 1. **小批量中样本数为1，文本序列为“想”“要”“有”“直”“升”“机”;每个时间步的输出层使用`softmax`,然后使用交叉熵作为损失函数**
> 2. **由于隐藏层中隐藏状态的循环计算,时间步3的输出$\boldsymbol{O}_3$取决于文本序列“想”“要”“有”**
> 3. **该序列的下一个词为“直”,时间步3的损失将取决于该时间步基于序列“想”“要”“有”生成下一个词的概率分布与该时间步的标签“直”**

![语言模型](img/6.2_rnn-train.svg)